# pybela tutorial 4: Monitor
This tutorial expects the `potentiometers` project to be running on Bela. If the Bela is connected to your laptop, you can run the cell below to copy the `potentiometers` code with the `Watcher` library onto your Bela:

In [ ]:
!rsync -rvL ../bela-code/potentiometers root@bela.local:Bela/projects

Then you can compile and run the project using either the IDE or by running the following command in the Terminal:
```bash
ssh root@bela.local "make -C Bela stop Bela PROJECT=potentiometers run" 
```
(Running this on a jupyter notebook will block the cell until the program is stopped on Bela.)

You will also need to connect two potentiometers to Bela analog inputs 0 and 1. Instructions on how to do so and some details on the Bela code are given in the notebook `1_Streamer-Bela-to-python-basics.ipynb`. The complete documentation for the pybela library can be found in [https://belaplatform.github.io/pybela/](https://belaplatform.github.io/pybela/).

This notebook is a tutorial for the Monitor class in the pybela python library. The monitor allows you to "take a look" at variables in your Bela code. By taking a look we mean either requesting a single value (*what value does `pot1` have right now?*) or sampling the value of a variable, that is, getting a value every number of frames (*can you tell me the value of `pot1` every 1000 frames?*). The monitor can be useful to calibrate sensors or, in general, debug your Bela code. 

The Monitor class inherits from the Streamer, so you can use it in a similar way, with the only difference that you will need to specify a monitoring period. Let's take a look at an example:

In [ ]:
import pandas as pd
from pybela import Monitor

As with the streamer, we need to instantiate the Monitor class and run the `connect()` method to establish the websocket connection with Bela. If the connection fails, make sure Bela is plugged in to your laptop and that the `potentiometer` project is running on Bela. 

In [ ]:
monitor = Monitor()
monitor.connect()

### Using asyncio to monitor data for a fixed amount of time

The monitor works in a very similar way to the Streamer but with a key difference: it doesn't continuously stream the values of the variables. Instead, it samples and streams these values periodically based on the given period. In this example, we will be monitoring two variables, `pot1` and `pot2` with a period 1000. 

As with the Streamer example, we can use asyncio to time the monitoring session:

In [ ]:
monitor.start_monitoring(variables= ["pot1", "pot2"], 
                         periods= [1000,2000])

monitor.wait(2)

monitored_values = monitor.stop_monitoring()

### Retrieving the data

`stop_monitoring` returns a dictionary with the monitored values and its timestamps. You can also access this data in `monitor.values`. Note that the timestamps are spaced by the period as we requested.

In [ ]:
df = pd.DataFrame(monitored_values["pot1"])
df.head()


In [ ]:
df = pd.DataFrame(monitored_values["pot2"])
df.head()

A note regarding the periods: in this example, the returned timestamps correspond to the analog frames elapsed in the Bela code. Let's take a look at the `render` loop in the `potentiometers` Bela project:

```cpp
void render(BelaContext *context, void *userData)
{
	for(unsigned int n = 0; n < context->audioFrames; n++) {
		if(gAudioFramesPerAnalogFrame && !(n % gAudioFramesPerAnalogFrame)) {
			
			uint64_t frames = context->audioFramesElapsed/gAudioFramesPerAnalogFrame + n/gAudioFramesPerAnalogFrame;
			Bela_getDefaultWatcherManager()->tick(frames); // watcher timestamps
			
			pot1 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot1Ch);
			pot2 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot2Ch);
			
		}
	}
}
```

AS you can see, we are "ticking" the Bela watcher once per every analog frame. When we request variable `pot1` with a period 1000, we will be getting the value of `pot1` every 1000 analog frames. For more advanced timestamping methods, you can check the `4_Sparse_timestamping.ipynb` notebook.

### Monitoring a fixed number of values
Alternatively, you can ask the monitor to monitor a variable for a fixed number of values. Another difference between the Streamer and the Monitor is that the Monitor will monitor exactly `n_values`. This is because in the monitor, the data buffers sent by Bela have only one timestamped value each, whilst in the Streamer, the buffers have a larger number of data points (which depends on the timestamping method and the data type). Let's see an example:

In [ ]:
monitored_data = monitor.monitor_n_values(variables= ["pot1", "pot2"],
                         periods= [1000,2000],
                         n_values= 10)

We can check that even when the periods for `pot1` and `pot2` where different, we monitored exactly 10 values for each:

In [ ]:
for var in ["pot1", "pot2"]:
    print(f"Monitored values for {var}: {len(monitored_data[var])}")

In [ ]:
df = pd.DataFrame(monitor.values["pot1"])

df.head()

In [ ]:
df = pd.DataFrame(monitor.values["pot2"])

df.head()

### Saving monitored data
You can also store the monitored values by passing `saving_enabled=True`  to either `start_monitoring()` or `monitor_n_values()`. 

In [ ]:
monitor.start_monitoring(variables= ["pot1", "pot2"],periods= [600,2000], saving_enabled=True)
monitor.wait(2)
monitor.stop_monitoring()

You can load the data using `load_data_from_file()`:

In [ ]:
pot1_saved_data = monitor.load_data_from_file("pot1_monitor.txt")

df = pd.DataFrame(pot1_saved_data)
df.head()

### Peeking at variables
You can also use the monitor to "peek" at variables, that is, requesting a single value.

In [ ]:
peeked_values = monitor.peek() # peeks at all the available variables (otherwise specify)
peeked_values["pot1"]